In [16]:
#!/usr/bin/env python

"""
Demonstrates plotting chromosome ideograms and genes (or any features, really)
using matplotlib.
1) Assumes a file from UCSC's Table Browser from the "cytoBandIdeo" table,
saved as "ideogram.txt". Lines look like this::
    #chrom  chromStart  chromEnd  name    gieStain
    chr1    0           2300000   p36.33  gneg
    chr1    2300000     5300000   p36.32  gpos25
    chr1    5300000     7100000   p36.31  gneg
2) Assumes another file, "ucsc_genes.txt", which is a BED format file
   downloaded from UCSC's Table Browser. This script will work with any
   BED-format file.
"""
import os as os
from matplotlib import pyplot as plt
from matplotlib.collections import BrokenBarHCollection
import pandas
import re as re

__author__ = 'Ryan Dale'
__source__ = 'https://www.biostars.org/p/147364/#147637'

# Here's the function that we'll call for each dataframe (once for chromosome
# ideograms, once for genes).  The rest of this script will be prepping data
# for input to this function
#
def chromosome_collections(df, y_positions, height,  **kwargs):
    """
    Yields BrokenBarHCollection of features that can be added to an Axes
    object.
    Parameters
    ----------
    df : pandas.DataFrame
        Must at least have columns ['chrom', 'start', 'end', 'color']. If no
        column 'width', it will be calculated from start/end.
    y_positions : dict
        Keys are chromosomes, values are y-value at which to anchor the
        BrokenBarHCollection
    height : float
        Height of each BrokenBarHCollection
    Additional kwargs are passed to BrokenBarHCollection
    """
    del_width = False
    if 'width' not in df.columns:
        del_width = True
        df['width'] = df['end'] - df['start']
    for chrom, group in df.groupby('chrom'):
        yrange = (y_positions[chrom], height)
        xranges = group[['start', 'width']].values
        yield BrokenBarHCollection(
            xranges, yrange, facecolors=group['colors'], **kwargs)
    if del_width:
        del df['width']


# Height of each ideogram
chrom_height = 1

# Spacing between consecutive ideograms
chrom_spacing = 1.5

# Height of the gene track. Should be smaller than `chrom_spacing` in order to
# fit correctly
gene_height = 1

# Padding between the top of a gene track and its corresponding ideogram
gene_padding = 0.1

# Width, height (in inches)
figsize = (10, 12)

# Decide which chromosomes to use
chromosome_list = ['chr%s' % i for i in list(range(1, 23)) + ['M', 'X', 'Y']]
chromosome_list = ['chr%s' % i for i in list(range(1, 23)) + ['X']]

# Keep track of the y positions for ideograms and genes for each chromosome,
# and the center of each ideogram (which is where we'll put the ytick labels)
ybase = 0
chrom_ybase = {}
gene_ybase = {}
chrom_centers = {}

# Iterate in reverse so that items in the beginning of `chromosome_list` will
# appear at the top of the plot
for chrom in chromosome_list[::-1]:
    chrom_ybase[chrom] = ybase
    chrom_centers[chrom] = ybase + chrom_height / 2.
    gene_ybase[chrom] = ybase - gene_height - gene_padding
    ybase += chrom_height + chrom_spacing

# Read in ideogram.txt, downloaded from UCSC Table Browser
ideo = pandas.read_table(
    '/home/pebert/Downloads/hg38_ideogram.bed',
    skiprows=1,
    names=['chrom', 'start', 'end', 'name', 'gieStain']
)

# Filter out chromosomes not in our list
ideo = ideo[ideo.chrom.apply(lambda x: x in chromosome_list)]

# Add a new column for width
ideo['width'] = ideo.end - ideo.start

# Colors for different chromosome stains
color_lookup = {
    'gneg': (1., 1., 1.),
    'gpos25': (.6, .6, .6),
    'gpos50': (.4, .4, .4),
    'gpos75': (.2, .2, .2),
    'gpos100': (0., 0., 0.),
    'acen': (.8, .4, .4),
    'gvar': (.8, .8, .8),
    'stalk': (.9, .9, .9),
}

# Add a new column for colors
ideo['colors'] = ideo['gieStain'].apply(lambda x: color_lookup[x])


# # Same thing for genes
# genes = pandas.read_table(
#     'ucsc_genes.txt',
#     names=['chrom', 'start', 'end', 'name'],
#     usecols=range(4))
# genes = genes[genes.chrom.apply(lambda x: x in chromosome_list)]
# genes['width'] = genes.end - genes.start
# genes['colors'] = '#2243a8'

mummer_cols = {'[S1]': 'reference_start',
               '[E1]': 'reference_end',
              '[S2]': 'query_start',
              '[E2]': 'query_end',
              '[LEN 1]': 'align_length_ref',
              '[LEN 2]': 'align_length_query',
              '[LEN R]': 'reference_length',
              '[LEN Q]': 'query_length',
              '[% IDY]': 'pct_identity',
              '[TAGS]': 'reference_id',
              '[QID]': 'query_id'}

# such a good tool and yet such an outdated output formatting...
# this makes me sad
mummer_header = '[S1]\t[E1]\t[S2]\t[E2]\t[LEN 1]\t[LEN 2]\t[% IDY]\t[LEN R]\t[LEN Q]\t[TAGS]\t[QID]'.split('\t')

title_filetype = {'cmp': 'Raw',
                 'tag-h1': 'Hap1',
                 'tag-h2': 'Hap2',
                 'tag-un': 'Untag',
                 'tag-h1-un': 'Hap1-Un',
                 'tag-h2-un': 'Hap2-Un'}

prettify_sample = {
    'HG00514.sequel2.pb-ccs': 'HG.514 PB-Sq2 CCS',
    'HG00514.sequel2.pb-clr': 'HG.514 PB-Sq2 CLR',
    'HG00514.sequel2.pb-clr-25': 'HG.514 PB-Sq2 CLR (0.25)'
}

inpath = '/MMCI/TM/scratch/pebert/diploid_assembly/output/assembly_analysis/mummer'
outpath = '/MMCI/TM/scratch/pebert/diploid_assembly/output/plots/alignment_ideograms'
os.makedirs(outpath, exist_ok=True)
coord_files = [os.path.join(inpath, f) for f in os.listdir(inpath) if f.endswith('coord.tab')]

for table in coord_files:
    if '_vs_' in table:
        continue
    output = os.path.join(outpath, os.path.basename(table).replace('coord.tab', 'aln_ideo.png'))
    if os.path.isfile(output):
        continue
    parts = os.path.basename(table).split('.')
    sample = '.'.join(parts[:2])
    mobj = re.search('tag[\-h12un]+', os.path.basename(table))
    assert mobj is not None, 'No haplotype detected'
    filetype = mobj.group(0)
    filetype = title_filetype[filetype]
    
    filename = os.path.basename(table)
    file_title = filename.replace('sequel2', 'sq2').replace('HG00514', 'HG514').replace('.coord.tab', '')
    file_title = file_title.replace('_to_', '\nmap to\n', 1)
    file_title = file_title.replace('pb-clr-25', 'CLR25').replace('pb-clr', 'CLR')
    file_title = file_title.replace('pb-ccs', 'CCS')
    
    # deliberately skip the header row as the query ID
    # is not part of it for whatever reason
    aln = pandas.read_table(table, skiprows=3+1, sep='\t',
                            index_col=False, names=mummer_header)
    new_cols = [mummer_cols[c] for c in aln.columns]
    aln.columns = new_cols
    aln['chrom'] = aln['reference_id']
    aln = aln[aln.chrom.apply(lambda x: x in chromosome_list)].copy()
    aln['start'] = aln['reference_start']
    aln['width'] = aln['align_length_ref']
    aln['colors'] = 'white'
    # better safe than sorry
    aln.sort_values(['chrom', 'start'], inplace=True)
    aln.reset_index(drop=True, inplace=True)
    if False:
        # simple alternating color scheme
        aln.loc[list(range(0, aln.shape[0], 2)), ['colors']] = 'dodgerblue'
        aln.loc[list(range(1, aln.shape[0], 2)), ['colors']] = 'orange'
    else:
        last_query_id = aln.at[0, 'query_id']
        last_chrom = aln.at[0, 'chrom']
        color_scheme = []
        c1, c2 = 'dodgerblue', 'orange'
        for row in aln.itertuples():
            if row.chrom != last_chrom:
                c1, c2 = 'dodgerblue', 'orange'
                last_chrom = row.chrom
                last_query_id = row.query_id
            if row.query_id != last_query_id:
                last_query_id = row.query_id
                c1, c2 = c2, c1
            color_scheme.append(c1)
        aln['colors'] = color_scheme

    fig = plt.figure(figsize=figsize)
    ax = fig.add_subplot(111)

    # Now all we have to do is call our function for the ideogram data...
    #print("adding ideograms...")
    for collection in chromosome_collections(ideo, chrom_ybase, chrom_height, edgecolor='black'):
        ax.add_collection(collection)

    # ...and the gene data
    #print("adding alignments...")
    for collection in chromosome_collections(
        aln, gene_ybase, gene_height, linewidths=0
    ):
        ax.add_collection(collection)

    text = 'Alignments: {}\n'.format(aln.shape[0])
    text += 'Median length: {} kbp\n'.format(int(round(aln['width'].median() / 1e3, 0)))
    text += 'Coverage: ~{}x'.format(round(aln['width'].sum() / (3 * 1e9), 2))
    
    txt = ax.text(0.6, 0.2, s=text, transform=ax.transAxes, fontdict={'fontsize': 14})
        
    # Axes tweaking
    ax.set_yticks([chrom_centers[i] for i in chromosome_list])
    ax.set_yticklabels(chromosome_list)
    ax.axis('tight')
    tt = ax.set_title('{}: {}'.format(filetype, file_title), fontsize=12)
    tt.set_position([0.5, 1.025])
    ax.set_xlabel('Genomic coordinate', fontsize=12)
    ax.set_ylabel('Chromosome', fontsize=14)
    
    fig.savefig(output, dpi=600, bbox_inches='tight', extra_artists=[txt])
    plt.close(fig)

